# Biosound data - Tidy up 
Prep the biosound data for use in the dashboard

To run this in Pycharm you need to set up a kernel that points to the venv:
`python -m ipykernel install --user --name=venv-osa`

Also please note that the data files and folders in this notebook point to a specific path a local Google Drive shared folder. This will not work for others - the paths will need updating for each person.

In [2]:
import pandas as pd
import numpy as np

import data_wrangler as dw
import os

In [15]:
# Set up some initial things
# Local path to the main "Processed Data" folder
local_data_folder = ('/Users/michelle/Library/CloudStorage/GoogleDrive-michelle@waveformanalytics.com/.shortcut'
                     '-targets-by-id/1QNPk7Z3Yb2uhHsHCf49pw7k_2-ivhIJg/Processed_Data')

# config_file = os.path.join(local_data_folder, 'BioSound_Datasets_MapApp.csv')
config_file = os.path.join(local_data_folder, 'BioSound_Datasets_MapApp_v1.csv')
df_config = pd.read_csv(config_file)

# Folder containing annotations data files
annotations_folder = os.path.join(local_data_folder, 'Annotations')

# Load the fish/annotations codes lookup table
df_codes = pd.read_csv("../shiny/data/fish_codes.csv")

# Satellite water class data
seascaper_folder = os.path.join(local_data_folder,"All_SeascapeR")

# Output file path for the duckdb file 
db_file = os.path.join(local_data_folder, "mbon4.duckdb")

In [4]:
# Take a look at the contents of the config file
df_config

,Dataset,short name,Source,Contact,Sampling Rate (kHz),Bit Rate,Recording Cycle,tz in file,tz local,Hydrophone Latitude,Hydrophone Longitude,Instrument Depth (m),Ecosystem Type,Selected Time Periods,Seascaper File,Annotations File
0,Caesar Creek,Biscayne Bay,University of Miami,Neil Hammerschlag/Abby Tinari,32,16,10 seconds every minute,America/New_York,America/New_York,25.396,-80.237,NaN,Mangrove,Feb - Mar 2019,CaesarCreek_SeascapeR_20190101_to_20190401.csv,NaN
1,Chuckchi Sea,Chukchi Sea,Oregon State University,Kate Stafford,16,16,25 minutes every hour,UTC,America/Nome,71.600,-161.500,30.0,Offshore,Jan - Feb 2019,ChuckChi_Sea_SeascapeR_20190101_to_20190401.csv,Chuckchi_Sea_Vessel_2019_02_ISO.csv
2,Gray's Reef,Gray's Reef,NEFSC-SanctSound,Tim Rowell,48,16,Continuous five hour files,UTC,America/New_York,31.396,-80.890,16.0,Offshore,Jan - Feb 2019,GraysReef_SeascapeR_20190101_to_20190401.csv,sanctsound_products_detections_gr01_sanctsound...
3,Key West,Key West,Florida Fish & Wildlife,Jess Keller,48,16,30 seconds every five minutes,America/New_York,America/New_York,24.442,-81.934,23.0,Coral Reef,Feb - Mar 2020,KeyWest_SeascapeR_20200101_to_20200401.csv,KW_Annotations.csv
4,May River,May River,University of S. Carolina Beaufort,Alyssa Marian/Eric Montie,80,16,2 minutes every hour,America/New_York,America/New_York,32.195,-80.792,4.5,Estuary,Jan - Mar 2019,MayRiver_SeascapeR_20190101_to_20190401.csv,MR_Annotations.csv
5,SanctSound-HI01,Olowalu,SanctSound - Hawaiian Islands Humpback NMS,Eden Zang,48,16,Continuous five hour files,UTC,Pacific/Honolulu,20.807,-156.655,59.7,Island/ Nearshore,Jan - Feb 2019,HI01_NMS_SeascapeR_20190101_to_20190401.csv,sanctsound_products_detections_hi01_sanctsound...
6,ONC-MEF,ONC,Ocean Networks Canada,Jasper Kanes,64,24,Continuous 5 minute files,UTC,America/Los_Angeles,47.949,-129.098,2189.0,Offshore,Jan - Feb 2019,ONC_MEF_SeascapeR_20190101_to_20190401.csv,ONC_MEF_Vessels_2019_02_ISO.csv
7,OOI-HYDBBA106,OOI,Ocean Observatories Initiative,Liz Ferguson,64,16,Continuous 5 minute files,UTC,America/Los_Angeles,44.637,-124.306,80.0,Oregon Shelf,Jan - Feb 2019,HYDBBA106_SeascapeR_20190101_to_20190401.csv,OOI_HYDBBA105_Vessel_2019_02_ISO.csv


## Annotations data

### Prep Key West annotations
Key west annotations are a bit different from the others. They're in a folder that contains several .txt files that need to first be merged and then copied into the same annotations folder as the other datasets.

In [5]:
KW_ANNOTATIONS_FOLDER = os.path.join(local_data_folder, 'Annotations/key-west-original')
KW_ANNOTATIONS_OUTFILE = os.path.join(local_data_folder, 'Annotations/KW_Annotations.csv')

# Run the function from data_wrangler to convert to the regular annotations format and move the file to the same location as the rest of the annotation files
df_kw_anno = dw.annotation_prep_kw_style(KW_ANNOTATIONS_FOLDER, KW_ANNOTATIONS_OUTFILE)

### Prep May River annotations data

May River annotations are also unique. They're stored in wide format in the "Master_Manual....xlsx" file. In this section we re-format so that it looks more like the Key West annotations and all the ship annotations (long-format).

In [6]:
MR_ANNOTATIONS_FILE = os.path.join(local_data_folder, 'Annotations/may-river-original/Master_Manual_14M_2h_011119_071619.xlsx')
MR_ANNOTATIONS_OUTFILE = os.path.join(local_data_folder, 'Annotations/MR_Annotations.csv')

df_mr_anno = dw.annotation_prep_mr_style(MR_ANNOTATIONS_FILE, MR_ANNOTATIONS_OUTFILE, df_codes)

## Index data
Prepare and load the data from the various acoustic index files and combine them all into a big dataframe. Also create a second dataframe that has all the same index values but normalized. 

In [7]:
# INDEX_DATA_FOLDER = os.path.join(local_data_folder, 'Revised_Indices_AllData_v2')
INDEX_DATA_FOLDER = os.path.join(local_data_folder, 'Original_Indices_AllData_v1')

df_aco0 = dw.prep_index_data(INDEX_DATA_FOLDER, normalize=False)
df_aco_norm0 = dw.prep_index_data(INDEX_DATA_FOLDER, normalize=True)

In [8]:
# Set time zones to local time
dw.update_time_zone(df_aco0, df_config)
dw.update_time_zone(df_aco_norm0, df_config)

,Date,Dataset,Sampling_Rate_kHz,FFT,Duration_sec,Thresholds_Hz,Filename,ZCR,MEANt,VARt,...,H_gamma,H_GiniSimpson,RAOQ,AGI,ROItotal,ROIcover,start_time,end_time,tz_file,tz_local
0,1/11/2019 11:20,May River,80,512,120,1500;8000;40000,May.River_20190111_112000.wav,0.478130,4.042254e-17,6.714753e-05,...,-0.442072,-0.064337,-0.406981,0.032916,0.022885,0.003838,2019-01-11 11:20:00-05:00,2019-01-11 11:40:00-05:00,America/New_York,America/New_York
1,1/11/2019 11:40,May River,80,512,120,1500;8000;40000,May.River_20190111_114000.wav,0.471607,-1.595750e-17,5.127475e-05,...,-0.379313,-0.035849,-0.364022,0.030485,0.021811,0.001971,2019-01-11 11:40:00-05:00,2019-01-11 12:00:00-05:00,America/New_York,America/New_York
2,1/11/2019 12:00,May River,80,512,120,1500;8000;40000,May.River_20190111_120000.wav,0.482468,-1.245578e-17,6.261245e-05,...,-0.226231,-0.009813,-0.247482,0.030061,0.003387,-0.001237,2019-01-11 12:00:00-05:00,2019-01-11 12:20:00-05:00,America/New_York,America/New_York
3,1/11/2019 12:20,May River,80,512,120,1500;8000;40000,May.River_20190111_122000.wav,0.561274,-4.512283e-19,7.451703e-05,...,-0.401797,-0.051974,-0.373290,0.035872,0.016854,-0.002375,2019-01-11 12:20:00-05:00,2019-01-11 12:40:00-05:00,America/New_York,America/New_York
4,1/11/2019 12:40,May River,80,512,120,1500;8000;40000,May.River_20190111_124000.wav,0.565080,-1.043466e-17,8.840571e-05,...,-0.474932,-0.076185,-0.438042,0.036860,0.012227,-0.004085,2019-01-11 12:40:00-05:00,2019-01-11 13:00:00-05:00,America/New_York,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294941,1/31/2019 22:20,May River,16,512,120,1500;8000;40000,May.River_20190131_222000.wav,-0.049953,1.638053e-17,3.259589e-06,...,0.074464,0.007079,0.127256,0.051507,0.002065,-0.006107,2019-01-31 22:20:00-05:00,2019-01-31 22:40:00-05:00,America/New_York,America/New_York
294942,1/31/2019 22:40,May River,16,512,120,1500;8000;40000,May.River_20190131_224000.wav,-0.028894,-1.560498e-17,1.105426e-06,...,-0.030413,-0.001342,0.006017,0.042243,0.004874,-0.004921,2019-01-31 22:40:00-05:00,2019-01-31 23:00:00-05:00,America/New_York,America/New_York
294943,1/31/2019 23:00,May River,16,512,120,1500;8000;40000,May.River_20190131_230000.wav,-0.036480,-1.835465e-18,2.550982e-07,...,0.220368,0.012509,0.251124,0.045430,0.007849,-0.004370,2019-01-31 23:00:00-05:00,2019-01-31 23:20:00-05:00,America/New_York,America/New_York
294944,1/31/2019 23:20,May River,16,512,120,1500;8000;40000,May.River_20190131_232000.wav,0.003339,-7.026941e-18,2.154163e-06,...,0.024728,0.005115,0.104338,0.047371,0.010988,0.001351,2019-01-31 23:20:00-05:00,2019-01-31 23:40:00-05:00,America/New_York,America/New_York


## Add the annotation information to index dataframes

Add new columns to the index dataframes for all possible indices for both presence and count.

In [9]:
unique_codes = np.unique(df_codes['code']).tolist()

# Add new columns that are named using unique_fish_codes. These will become the 
df_aco = dw.add_new_columns(df_aco0, unique_codes)
df_aco_norm = dw.add_new_columns(df_aco_norm0, unique_codes)

In [10]:
df_aco_anno = dw.add_annotations_to_df(df_aco, df_config, df_codes, annotations_folder)
df_aco_norm_anno = dw.add_annotations_to_df(df_aco_norm, df_config, df_codes, annotations_folder)

/Users/michelle/Documents/WORKSPACE/osa-mbon/python-prep/data_wrangler.py:274: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, df_sub_with_presence], axis=0)
/Users/michelle/Documents/WORKSPACE/osa-mbon/python-prep/data_wrangler.py:274: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, df_sub_with_presence], axis=0)


## Prep Seascaper Data


In [11]:
df_seascaper = dw.prep_seascaper_data(seascaper_folder, df_config)

## Create DuckDB

Save the main pandas dataframes to a duckdb file.

In [19]:
# Prep the dataframe list for export
df_dict = {
    "t_aco2": df_aco_anno,
    "t_aco_norm2": df_aco_norm_anno,
    "t_seascaper": df_seascaper,
    "t_fish_keywest": df_kw_anno,
    "t_fish_mayriver": df_mr_anno
}

dw.duckdb_export(db_file, df_dict)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [18]:
df_mr_anno.columns

Index(['start_time', 'end_time', 'Labels', 'is_present'], dtype='object')